In [1]:
import os
from pathlib import Path

import emv_tools
import emv_tools.ffi.scipion as scipion
from emv_tools.ffi.blocres import blocres
from emv_tools.scipion_bridge.proxy import OutputInfo

from utils import download as D
emv_tools.scipion_bridge.environment.configure_default_env()

## Download Data

In [15]:
DOWNLOAD_PATH = Path("data/downloaded")
EMDB_ENTRY = 11004

In [16]:
metadata = D.download_emdb_metadata(entry_id=EMDB_ENTRY)

os.makedirs(DOWNLOAD_PATH, exist_ok=True)

emdb_map = D.download_emdb_map(EMDB_ENTRY, DOWNLOAD_PATH)
map_1, map_2 = D.download_halfmaps(EMDB_ENTRY, DOWNLOAD_PATH)
pdb_model = D.download_pdb_model("6yys", DOWNLOAD_PATH) # metadata.pdb_id

metadata

EMDBMetadata(pdb_id='6f6w', resolution=3.08, sampling=0.8310999, size=360, org_x=0, org_y=0, org_z=0)

## Create Volume from PDB

In [17]:
volume = scipion.xmipp_volume_from_pdb(
    pdb_model,
    OutputInfo(None),
    center_pdb="-v 0",
    sampling=metadata.sampling,
    size=metadata.size,
).reassign("vol")

volume = scipion.xmipp_volume_align(
    embdb_map=str(emdb_map),
    volume=volume,
    local=True
)

[XMIPP] scipion run xmipp_volume_from_pdb -i data/downloaded/pdb6f6w.ent -o /tmp/tmp3fepgkyd --centerPDB -v 0 --sampling 0.8310999 --size 360
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_volume_align --i1 data/downloaded/emd_11004.map --i2 /tmp/tmp3fepgkyd.vol --local --apply /tmp/tmpjsx729c2.vol
1 (1,0,0,0,0,1,0,0,0)--->-0.391743
   (1,0,***5.59687,0,0,1,0,0,0)--->-0.406291
   (1,0,5.59687,***0.631204,0,1,0,0,0)--->-0.409095
   (1,0,5.59687,0.631204,***-0.330452,1,0,0,0)--->-0.409156
   (1,0,5.59687,0.631204,-0.330452,***1.13619,0,0,0)--->-0.423265
   (1,0,5.59687,0.631204,-0.330452,1.13619,***6.0142,0,0)--->-0.437111
   (1,0,5.59687,0.631204,-0.330452,1.13619,6.0142,***0.335609,0)--->-0.437246
   (1,0,5.59687,0.631204,-0.330452,1.13619,6.0142,0.335609,***-6.53256)--->-0.449789
2 (1,0,5.59687,0.631204,-0.330452,1.13619,6.0142,0.335609,-6.53256)--->-0.449789
   (1,0,***-1.14411,0.631204,-0.330452,1.13619,6.0142,0.335609,-6.53256)--->-0.457786
   (1,0,-1.14

### Mask from Volume

In [18]:
mask = scipion.xmipp_transform_threshold(
    volume, select="below 0.02", substitute="binarize"
)
mask = scipion.xmipp_transform_morphology(mask, binary_operation="dilation", size=2)

mask

[XMIPP] scipion run xmipp_transform_threshold -i /tmp/tmpjsx729c2.vol -o /tmp/tmp254mgspw.vol --select below 0.02 --substitute binarize
Input File: /tmp/tmpjsx729c2.vol
Output File: /tmp/tmp254mgspw.vol
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_transform_morphology -i /tmp/tmp254mgspw.vol -o /tmp/tmptld0e1vn.vol --binaryOperation dilation --size 2
Input File: /tmp/tmp254mgspw.vol
Output File: /tmp/tmptld0e1vn.vol
Initially the image has 1.12892e+06 pixels set to 1
Finally the image has 1.47323e+06 pixels set to 1
Xmipp command detected
Scipion v3.7.1 - Eugenius


Output()

In [19]:
def apply_sampling(data, sampling):
    data = scipion.xmipp_image_convert(data)
    scipion.xmipp_image_header(data, sampling=sampling)

    return data

volume = apply_sampling(volume, metadata.sampling)
mask = apply_sampling(mask, metadata.sampling)

[XMIPP] scipion run xmipp_image_convert -i /tmp/tmpjsx729c2.vol -o /tmp/tmp7xej8vov.mrc -t vol
Input File: /tmp/tmpjsx729c2.vol
Output File: /tmp/tmp7xej8vov.mrc
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_image_header -i /tmp/tmp7xej8vov.mrc -s 0.8310999
Input File: /tmp/tmp7xej8vov.mrc
New sampling rate (Angstrom) = 0.8311
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_image_convert -i /tmp/tmptld0e1vn.vol -o /tmp/tmpsp9jmkxn.mrc -t vol
Input File: /tmp/tmptld0e1vn.vol
Output File: /tmp/tmpsp9jmkxn.mrc
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_image_header -i /tmp/tmpsp9jmkxn.mrc -s 0.8310999
Input File: /tmp/tmpsp9jmkxn.mrc
New sampling rate (Angstrom) = 0.8311
Xmipp command detected
Scipion v3.7.1 - Eugenius


## Compute BlocRes

In [30]:
blocres_half = blocres(
    str(map_1),
    str(map_2),
    mask=mask,
    sampling=f"{metadata.sampling},{metadata.sampling},{metadata.sampling}",
    box=int(metadata.resolution * 5),
    cutoff=0.5,
    step=1,
)

[BlocRes] /home/max/scipion3/software/em/bsoft/bin/blocres  -Mask /tmp/tmpsp9jmkxn.mrc -sampling 0.8310999,0.8310999,0.8310999 -box 15 -cutoff 0.5 -nofill -smooth -pad 1 -step 1 -maxresolution 0.5 -verbose 0 data/downloaded/emd_11004_half_map_1.map data/downloaded/emd_11004_half_map_2.map /tmp/tmpccizbqpg.map


In [31]:
blocres_model = blocres(
    str(emdb_map),
    volume,
    mask=mask,
    sampling=f"{metadata.sampling},{metadata.sampling},{metadata.sampling}",
    box=int(metadata.resolution * 5),
    cutoff=0.67,
    step=1,
)

[BlocRes] /home/max/scipion3/software/em/bsoft/bin/blocres  -Mask /tmp/tmpsp9jmkxn.mrc -sampling 0.8310999,0.8310999,0.8310999 -box 15 -cutoff 0.67 -nofill -smooth -pad 1 -step 1 -maxresolution 0.5 -verbose 0 data/downloaded/emd_11004.map /tmp/tmp7xej8vov.mrc /tmp/tmpq8lhi5u1.map


## Compute FSCQ

In [32]:
fscq_diff = scipion.xmipp_image_operate(blocres_model, minus=blocres_half)
atomic_model = scipion.xmipp_pdb_label_from_volume(
    pdb=str(pdb_model),
    volume=fscq_diff,
    mask=mask,
    sampling=metadata.sampling,
    origin="%f %f %f" % (metadata.org_x, metadata.org_y, metadata.org_z),
)

[XMIPP] scipion run xmipp_image_operate -i /tmp/tmpq8lhi5u1.map -o /tmp/tmp1kyt5b41.vol --minus /tmp/tmpccizbqpg.map
Input File: /tmp/tmpq8lhi5u1.map
Output File: /tmp/tmp1kyt5b41.vol
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_pdb_label_from_volume -o /tmp/tmpliig36sx.atom.pdb --pdb data/downloaded/pdb6f6w.ent --vol /tmp/tmp1kyt5b41.vol --mask /tmp/tmpsp9jmkxn.mrc --sampling 0.8310999 --origin 0.000000 0.000000 0.000000
PDB file:           data/downloaded/pdb6f6w.ent
Output:       /tmp/tmpliig36sx.atom.pdb
Sampling rate:        0.8311
Origin:               0.000000 0.000000 0.000000 
Radius:               0.8
mean value: = 7.27387
absolute mean value: = 7.47501
Xmipp command detected
Scipion v3.7.1 - Eugenius


In [33]:
with open(atomic_model.path) as f:
    print(f.read())

ATOM      1 N    MET A   1     123.028 106.822 198.843  5.43155.56           N  
ATOM      2 CA   MET A   1     124.395 107.023 199.400  5.53156.57           C  
ATOM      3 C    MET A   1     124.876 108.441 199.059  5.54153.56           C  
ATOM      4 O    MET A   1     125.593 108.608 198.049  6.39152.68           O  
ATOM      5 CB   MET A   1     125.364 105.973 198.841  5.58160.72           C  
ATOM      6 CG   MET A   1     126.727 105.974 199.512  7.13163.37           C  
ATOM      7 SD   MET A   1     127.724 104.534 199.050  5.97165.58           S  
ATOM      8 CE   MET A   1     128.908 104.506 200.395  1.48165.70           C  
ATOM      9 N    LEU A   2     124.476 109.424 199.873  5.95152.45           N  
ATOM     10 CA   LEU A   2     124.904 110.847 199.751  5.92152.62           C  
ATOM     11 C    LEU A   2     126.301 111.012 200.360  6.08153.91           C  
ATOM     12 O    LEU A   2     126.738 110.112 201.109  6.42154.17           O  
ATOM     13 CB   LEU A   2  

In [48]:
scipion.xmipp_transform_threshold(
    fscq_diff, select="below 0.02", substitute="binarize"
)

[XMIPP] scipion run xmipp_transform_threshold -i /tmp/tmp1kyt5b41.vol -o /tmp/tmptq8pzk99.vol --select below 0.02 --substitute binarize
Input File: /tmp/tmp1kyt5b41.vol
Output File: /tmp/tmptq8pzk99.vol
Xmipp command detected
Scipion v3.7.1 - Eugenius


Output()

In [41]:
!scipion run xmipp_transform_threshold

Scipion v3.7.1 - Eugenius
Xmipp command detected
This method will return the plugin class in the future. Please use getPluginModule instead
PROGRAM
   xmipp_transform_threshold
USAGE
   Threshold volumes and images 
SEE ALSO
   transform_mask, transform_morphology
OPTIONS

   -i, --input <input_file>
          Input file: metadata, stack, volume or image. 
   [-o, --output <output_file=>]
          Output file: metadata, stack, volume or image. 
   [--oroot <root=>]
          Rootname of output individual images. 
          Output image format can be set adding extension after rootname as ":ext". 
   --select <mode>
           Select pixels meeting 
      where <mode> can be:
        abs_below <th>
           Absolute value below a threshold 
        below <th>
           Below a threshold 
        above <th>
           Above a threshold 
   --substitute <substitutionMode=value>
           Substitute selected pixels by 
      where <substitutionMode> can be:
        binarize
          